In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [15]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, padding='post', truncating='post', maxlen=40)

X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_sequences, padding='post', truncating='post', maxlen=40)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, padding='post', truncating='post', maxlen=40)

In [16]:
len(word_index)

23922

In [17]:
from tensorflow import keras
from tensorflow.keras import layers

In [18]:
embedding_dim = 64
vocab_size = len(word_index) + 1

model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          1531072   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,642,561
Trainable params: 1,642,561
Non-

In [20]:
from tensorflow.keras import callbacks, metrics

In [35]:
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4), 
    loss='binary_crossentropy', 
    metrics=METRICS
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-4,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

In [22]:
import tensorflow as tf

In [36]:
history = model.fit(
    tf.constant(X_train_padded), 
    tf.constant(y_train), 
    validation_data=(tf.constant(X_val_padded), tf.constant(y_val)),
    batch_size=32,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20
656/656 [==============================] - 19s 22ms/step - loss: 0.2296 - accuracy: 0.9157 - precision: 0.9176 - recall: 0.9135 - val_loss: 0.3373 - val_accuracy: 0.8667 - val_precision: 0.8521 - val_recall: 0.8874
Epoch 2/20
656/656 [==============================] - 13s 20ms/step - loss: 0.1376 - accuracy: 0.9531 - precision: 0.9533 - recall: 0.9529 - val_loss: 0.3688 - val_accuracy: 0.8674 - val_precision: 0.8277 - val_recall: 0.9279
Epoch 3/20
656/656 [==============================] - 14s 21ms/step - loss: 0.0789 - accuracy: 0.9763 - precision: 0.9767 - recall: 0.9758 - val_loss: 0.4366 - val_accuracy: 0.8645 - val_precision: 0.8157 - val_recall: 0.9417
Epoch 4/20
656/656 [==============================] - 16s 25ms/step - loss: 0.0439 - accuracy: 0.9879 - precision: 0.9881 - recall: 0.9877 - val_loss: 0.4373 - val_accuracy: 0.8745 - val_precision: 0.8422 - val_recall: 0.9217
Epoch 4: early stopping
